In [1]:
import pandas as pd
import os


dirori = ''

In [2]:
# tickets extraction and rename head
df_tickets = pd.read_csv('', encoding='latin-1')
df_inc_sla = pd.read_csv('', encoding='latin-1')

def rename_columns(df):
    for col in df.columns:
        df = df.rename(columns={col:col.upper().replace('INC_','')})
        col = col.upper().replace('INC_','')
        df = df.rename(columns={col:col.replace('TASKSLATABLE_','')})
        col = col.replace('TASKSLATABLE_','')
        df = df.rename(columns={col:col.replace('SYS_','')})
    return df

df_tickets = rename_columns(df_tickets)
df_inc_sla = rename_columns(df_inc_sla)

In [3]:
df_inc_sla = df_inc_sla[((df_inc_sla['STATE'].isin(['New', 'Work in Progress', 'On Hold']))
                         & (df_inc_sla['SLA'].str.contains(''))
                         & (df_inc_sla['ACTIVE']==True))].sort_values(by='NUMBER').reset_index(drop=True)
df_inc_sla['BUSINESS_PERCENTAGE'] = df_inc_sla['BUSINESS_PERCENTAGE'].str.replace(',', '')
df_inc_sla['BUSINESS_PERCENTAGE'] = pd.to_numeric(df_inc_sla['BUSINESS_PERCENTAGE'])/100
df_inc_sla = df_inc_sla[['NUMBER' ,'BUSINESS_PERCENTAGE' ,'SLA']]
#df_inc_sla

In [4]:
from datetime import datetime, timedelta, date

try:
    for tkt in df_tickets['NUMBER']:
        match = df_old_join[df_old_join['NUMBER'] == tkt]
        if match.empty:
            df_tickets.loc[df_tickets['NUMBER']==tkt,'C_FIRST_ASSIGN_ESTIMATE'] = datetime.today()
        else:
            df_tickets.loc[df_tickets['NUMBER']==tkt,'C_FIRST_ASSIGN_ESTIMATE'] = df_old_join.loc[df_old_join['NUMBER']==tkt,'C_FIRST_ASSIGN_ESTIMATE']
except Exception as e:
    df_tickets['C_FIRST_ASSIGN_ESTIMATE'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
    print('Error')

Error


In [5]:
# consultor extraction and rename head
df_consultor = pd.read_excel('')
for col in df_consultor.columns:
    df_consultor = df_consultor.rename(columns={col:col.upper()})

In [6]:
# Bulding a Day Calendar with the paremeters already defined before
from dateutil.relativedelta import relativedelta

anoini = date.today().year - 1
anofim = date.today().year + 1

dateiniday = datetime(anoini, 1, 1)
datefimday = datetime(anofim, 12, 31)
dtl30d = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0) + relativedelta(days=-30)

current_year = datetime.today().year
current_month = datetime.today().month
current_date = datetime.today()
l12m = current_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0) + relativedelta(months=-12)
lm_1_ini = current_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0) + relativedelta(months=0)
lm_1_fim = current_date.replace(day=1, hour=0, minute=0, second=0, microsecond=0) + relativedelta(months=+1) + relativedelta(days=-1)

date_range_day = pd.date_range(start=dateiniday, end=datefimday, freq='D')

df_calendar_day = pd.DataFrame({'DATE': date_range_day})

df_calendar_day['DATE'] = pd.to_datetime(df_calendar_day['DATE']) #, format='%Y-%m-%d'

df_calendar_day['FLAG_CURRENT_MONTH_DATE'] = False
df_calendar_day['FLAG_CURRENT_YEAR_DATE'] = False
df_calendar_day['FLAG_L12M_DATE'] = False
df_calendar_day['FLAG_LM1_DATE'] = False
df_calendar_day['FLAG_L30D'] = False
df_calendar_day.loc[(df_calendar_day['DATE'].dt.year == current_year) &
                (df_calendar_day['DATE'].dt.month  == current_month),
                 'FLAG_CURRENT_MONTH_DATE'] = True
df_calendar_day.loc[(df_calendar_day['DATE'].dt.year == current_year),
                 'FLAG_CURRENT_YEAR_DATE'] = True
df_calendar_day.loc[(df_calendar_day['DATE'] >= l12m) & (df_calendar_day['DATE'] <= current_date),
                 'FLAG_L12M_DATE'] = True
df_calendar_day.loc[(df_calendar_day['DATE'] >= lm_1_ini) & (df_calendar_day['DATE'] <= lm_1_fim), 'FLAG_LM1_DATE'] = True
df_calendar_day.loc[(df_calendar_day['DATE'] >= dtl30d) & (df_calendar_day['DATE'] <= current_date), 'FLAG_L30D'] = True
df_calendar_day['YEAR_MONTH'] = df_calendar_day['DATE'].dt.to_period('M')
df_calendar_day['YEAR'] = df_calendar_day['DATE'].dt.year
df_calendar_day['MONTH'] = df_calendar_day['DATE'].dt.month
df_calendar_day['DAY'] = df_calendar_day['DATE'].dt.day
df_calendar_day['MONTH_ABBR'] = df_calendar_day['DATE'].dt.strftime('%b')
df_calendar_day = df_calendar_day.sort_values(by='DATE', ascending = True)
df_calendar_day = df_calendar_day.reset_index()

df_calendar_day.to_csv(f'', sep=';', index=False)

In [7]:
df_tickets_v3 = pd.merge(df_tickets, df_consultor, left_on='ASSIGNED_TO', right_on='NAME_KEY', how = 'left')

In [8]:
# Department Border
df_tickets_v3['DEPARTMENT'] = 'SAP'
df_tickets_v3['DEPARTMENT'][df_tickets_v3['BORDER'].isin(['Tableau','Non-SAP'])] = 'Non-SAP'
df_tickets_v3['DEPARTMENT'][df_tickets_v3['BORDER'].isin(['Salesforce'])] = 'Salesforce'

In [9]:
dateini = datetime(anoini, 1, 1)
datefim = datetime(anofim, 12, 31, 23, 59,59)

df_tickets = df_tickets_v3.copy()
df_tickets = df_tickets[pd.to_datetime(df_tickets['OPENED_AT']) >= dateini]
current_year = datetime.now().year
current_month = datetime.now().month
l12m = datetime.today() + relativedelta(months=-12)
l12m = datetime(l12m.year, l12m.month, 1)

# Flag date closed
df_tickets['FLAG_CURRENT_MONTH_CLOSED'] = False
df_tickets['FLAG_CURRENT_YEAR_CLOSED'] = False
df_tickets['FLAG_L12M_CLOSED'] = False
df_tickets.loc[(pd.to_datetime(df_tickets['CLOSED_AT']).dt.year == current_year) &
                (pd.to_datetime(df_tickets['CLOSED_AT']).dt.month == current_month),
                 'FLAG_CURRENT_MONTH_CLOSED'] = True
df_tickets.loc[(pd.to_datetime(df_tickets['CLOSED_AT']).dt.year == current_year),
                 'FLAG_CURRENT_YEAR_CLOSED'] = True
df_tickets.loc[(pd.to_datetime(df_tickets['CLOSED_AT']) >= l12m),
                 'FLAG_L12M_CLOSED'] = True

# Flag date opened
df_tickets['FLAG_CURRENT_MONTH_OPENED'] = False
df_tickets['FLAG_CURRENT_YEAR_OPENED'] = False
df_tickets['FLAG_L12M_OPENED'] = False
df_tickets.loc[(pd.to_datetime(df_tickets['OPENED_AT']).dt.year == current_year) &
                (pd.to_datetime(df_tickets['OPENED_AT']).dt.month == current_month),
                 'FLAG_CURRENT_MONTH_OPENED'] = True
df_tickets.loc[(pd.to_datetime(df_tickets['OPENED_AT']).dt.year == current_year),
                 'FLAG_CURRENT_YEAR_OPENED'] = True
df_tickets.loc[(pd.to_datetime(df_tickets['OPENED_AT']) >= l12m),
                 'FLAG_L12M_OPENED'] = True

# Flag date opened
df_tickets['FLAG_CURRENT_MONTH_RESOLVED'] = False
df_tickets['FLAG_CURRENT_YEAR_RESOLVED'] = False
df_tickets['FLAG_L12M_RESOLVED'] = False
df_tickets.loc[(pd.to_datetime(df_tickets['RESOLVED_AT']).dt.year == current_year) &
                (pd.to_datetime(df_tickets['RESOLVED_AT']).dt.month == current_month),
                 'FLAG_CURRENT_MONTH_RESOLVED'] = True
df_tickets.loc[(pd.to_datetime(df_tickets['RESOLVED_AT']).dt.year == current_year),
                 'FLAG_CURRENT_YEAR_RESOLVED'] = True
df_tickets.loc[(pd.to_datetime(df_tickets['RESOLVED_AT']) >= l12m),
                 'FLAG_L12M_RESOLVED'] = True

In [10]:
# REMOVE DUPLICATE
#df_tickets.drop(['STAGE', 'BUSINESS_PERCENTAGE'], axis=1, inplace=True)

df_tickets.drop_duplicates(inplace=True)
#print(df_tickets.groupby(['NUMBER'])['NUMBER'].count())
#print(df_tickets['NUMBER'].count())

mask = pd.to_datetime(df_tickets['RESOLVED_AT']).dt.date > pd.to_datetime(df_tickets['CLOSED_AT']).dt.date
df_tickets.loc[mask, 'RESOLVED_AT'] = df_tickets.loc[mask, 'CLOSED_AT']

In [11]:
# Create OPENED Table
df_opened = df_tickets[(df_tickets['OPENED_AT'] >= "2021-12-31") &
                        (~df_tickets['OPENED_AT'].isna())].copy()
df_opened['REFERENCE_AT_DATETIME'] = df_opened['OPENED_AT'] #df_opened['REFERENCE_AT_DATE'] = pd.to_datetime(df_opened['REFERENCE_AT_DATE'])
df_opened['STATUS_MONTH'] = 'OPENED'
df_opened['COUNT_TICKETS'] = 1
df_opened['YTD'] = ['YTD' if datetime.now().year == pd.to_datetime(data).year
                    else (pd.to_datetime(data).year - datetime.now().year) 
                    for data in df_opened['REFERENCE_AT_DATETIME']]

# Create Closed Table
df_closed = df_tickets[((df_tickets['RESOLVED_AT'] >= "2021-12-31") |
                        (df_tickets['CLOSED_AT'] >= "2021-12-31")) &
                        (~df_tickets['RESOLVED_AT'].isna()) &
                        (df_tickets['STATE'].str.upper() != 'CANCELED')].copy()
df_closed['REFERENCE_AT_DATETIME'] = df_closed['RESOLVED_AT'] #df_closed['REFERENCE_AT_DATE'] = pd.to_datetime(df_closed['REFERENCE_AT_DATE'])
df_closed['STATUS_MONTH'] = 'CLOSED'
df_closed['COUNT_TICKETS'] = 1
df_closed['YTD'] = ['YTD' if datetime.now().year == pd.to_datetime(data).year
                    else (pd.to_datetime(data).year - datetime.now().year) 
                    for data in df_closed['REFERENCE_AT_DATETIME']]

# Create CANCELED Table
df_canceled = df_tickets[(df_tickets['CLOSED_AT'] >= "2021-12-31") &
                        (df_tickets['STATE'].str.upper() == 'CANCELED')].copy()
df_canceled['REFERENCE_AT_DATETIME'] = df_canceled['CLOSED_AT'] #df_closed['REFERENCE_AT_DATE'] = pd.to_datetime(df_closed['REFERENCE_AT_DATE'])
df_canceled['STATUS_MONTH'] = 'CANCELED'
df_canceled['COUNT_TICKETS'] = 1
df_canceled['YTD'] = ['YTD' if datetime.now().year == pd.to_datetime(data).year
                    else (pd.to_datetime(data).year - datetime.now().year) 
                    for data in df_canceled['REFERENCE_AT_DATETIME']]
# Create BACKLOG Table

df_backlog = df_tickets[(df_tickets['RESOLVED_AT'] >= (pd.to_datetime(df_tickets['OPENED_AT']) + pd.offsets.DateOffset(days=1)).dt.to_period('D').dt.to_timestamp()) |
                        (df_tickets['CLOSED_AT'] >= (pd.to_datetime(df_tickets['OPENED_AT']) + pd.offsets.DateOffset(days=1)).dt.to_period('D').dt.to_timestamp()) &
                        (~df_tickets['CLOSED_AT'].isna() &
                        df_tickets['RESOLVED_AT'].isna()) |
                        (df_tickets['CLOSED_AT'].isna() &
                        df_tickets['RESOLVED_AT'].isna())].copy()

df_backlog_xpto = df_tickets[~df_tickets['STATE'].isin(['Closed','Canceled','Resolved'])]

df_backlog['REFERENCE_AT_DATETIME'] = df_backlog['OPENED_AT'] #df_closed['REFERENCE_AT_DATE'] = pd.to_datetime(df_closed['REFERENCE_AT_DATE'])
df_backlog['STATUS_MONTH'] = 'BACKLOG'
df_backlog['COUNT_TICKETS'] = 1
df_backlog['YTD'] = ['YTD' if datetime.now().year == pd.to_datetime(data).year
                    else (pd.to_datetime(data).year - datetime.now().year) 
                    for data in df_backlog['REFERENCE_AT_DATETIME']]

#df_backlog = df_backlog.head(50)

# LOOK FOR BACKLOG LINES WITH MORE THAN 1 MONTH OF DIFFERENCE BETWEEN RESOLVED/CLOSED AND OPENED TO INPUT A NEW LINE INTO THE GAP MONTH 
def dup_backlog_line(df):
    x = 0
    new_series = []
    for idx, row in df.iterrows():
        x = int(row['datediff'] - 1)
        for i in range(x):
            row['REFERENCE_AT_DATETIME'] = (pd.to_datetime(row['OPENED_AT']) + pd.offsets.DateOffset(days=i+1)).to_period('D').to_timestamp()
            #print(str(row['CREATED_AT']) + ' - ' + str(pd.offsets.DateOffset(days=i)) + ' - ' +str(pd.to_datetime(row['CREATED_AT']) + pd.offsets.DateOffset(days=i)))
           #print(str(row['NUMBER']) + ' - ' + str(row['OPENED_AT']) + ' - ' + str(row['REFERENCE_AT_DATETIME']) + ' - ' +  str(row['datediff']) + ' - ' + str(i))
            new_series.append(row.copy())
    return new_series

df_backlog_date_dif = df_backlog.copy()
df_backlog_date_dif['dateClosedOpened'] = df_backlog_date_dif['RESOLVED_AT']
df_backlog_date_dif.loc[df_backlog_date_dif['dateClosedOpened'].isna(),'dateClosedOpened'] = df_backlog_date_dif['CLOSED_AT']
df_backlog_date_dif.loc[df_backlog_date_dif['dateClosedOpened'].isna(),'dateClosedOpened'] = datetime.today().date()
df_backlog_date_dif['datediff'] = list(map(lambda x, y: (x-y).days, 
                                           pd.to_datetime(df_backlog_date_dif['dateClosedOpened']).dt.date, 
                                           pd.to_datetime(df_backlog_date_dif['OPENED_AT']).dt.date))

mask_backlog_today = ~df_backlog_date_dif['NUMBER'].isin(df_closed['NUMBER']) & ~df_backlog_date_dif['NUMBER'].isin(df_canceled['NUMBER'])
df_backlog_date_dif.loc[mask_backlog_today, 'datediff'] = df_backlog_date_dif.loc[mask_backlog_today, 'datediff'] + 1
df_backlog_date_dif = df_backlog_date_dif[df_backlog_date_dif['datediff']>1].copy()
df_auxiliar = pd.DataFrame(dup_backlog_line(df_backlog_date_dif))

#print(df_backlog_date_dif.loc[df_backlog_date_dif['NUMBER']=='INC0807869',['NUMBER','datediff']])
#print('')

#df_auxiliar = pd.concat([df_auxiliar, df_backlog_date_dif], axis=0)
df_auxiliar.drop(['dateClosedOpened','datediff'], axis=1, inplace=True)

df_backlog  = pd.concat([df_backlog, df_auxiliar], axis=0)


# UNION
df_daily_incidents = pd.concat([df_opened, df_closed, df_canceled, df_backlog], axis=0, ignore_index=True)

# REFERENCE_AT_DATETIME REFERENCE_AT_DATE
df_daily_incidents['REFERENCE_AT_DATETIME'] = pd.to_datetime(df_daily_incidents['REFERENCE_AT_DATETIME'])
df_daily_incidents['REFERENCE_AT_DATE'] = df_daily_incidents['REFERENCE_AT_DATETIME'].dt.date

In [12]:
# Flags REFERENCE_AT_DATE
df_daily_incidents['FLAG_CURRENT_MONTH_REFERENCE'] = False
df_daily_incidents['FLAG_CURRENT_YEAR_REFERENCE'] = False
df_daily_incidents['FLAG_L12M_REFERENCE'] = False
df_daily_incidents.loc[(df_daily_incidents['REFERENCE_AT_DATETIME'].dt.year == current_year) &
                (df_daily_incidents['REFERENCE_AT_DATETIME'].dt.month  == current_month),
                 'FLAG_CURRENT_MONTH_REFERENCE'] = True
df_daily_incidents.loc[(df_daily_incidents['REFERENCE_AT_DATETIME'].dt.year == current_year),
                 'FLAG_CURRENT_YEAR_REFERENCE'] = True
df_daily_incidents.loc[(df_daily_incidents['REFERENCE_AT_DATETIME'] >= l12m),
                 'FLAG_L12M_REFERENCE'] = True

In [13]:
current_date_dt = current_date.date()

#print(df_daily_incidents)

# Drop backlog closed on the last day
#mask_list_closed_last_day = ((df_daily_incidents['STATUS_MONTH']=='CLOSED') & 
#                             (df_daily_incidents['REFERENCE_AT_DATE']==current_date_dt))
#list_closed_last_day = df_daily_incidents[mask_list_closed_last_day]['NUMBER']
#mask_list_backlog = (df_daily_incidents['NUMBER'].isin(list_closed_last_day)
#                       & (df_daily_incidents['STATUS_MONTH'] == 'BACKLOG')
#                       & (df_daily_incidents['CLOSED_AT']==current_date_dt))
#df_daily_incidents = df_daily_incidents[~mask_list_backlog]

In [14]:
# CLASS AGING
df_daily_incidents['FLAG_OPENED'] = 'N'
df_daily_incidents.loc[(df_daily_incidents['STATE'].isin(['Work in Progress', 'On Hold', 'New']))
                       & (df_daily_incidents['REFERENCE_AT_DATE'] == current_date_dt), 'FLAG_OPENED'] = 'Y'

df_daily_incidents['AGING'] = [(pd.Timestamp(rd).date() - pd.Timestamp(op).date()) if fl == 'Y'
                               else (pd.Timestamp(rd).date() - pd.Timestamp(op).date())
                                for op, rd, fl in zip(df_daily_incidents['OPENED_AT'], df_daily_incidents['REFERENCE_AT_DATE'], 
                                          df_daily_incidents['FLAG_OPENED'])]
df_daily_incidents['AGING'] = (df_daily_incidents['AGING'].dt.days)

def class_aging(df):
    df_class = []
    for ag in df:
        if ag <= 5:
            df_class.append('<= 5')
        elif ag <= 10:
            df_class.append('<= 10')
        elif ag <= 15:
            df_class.append('<= 15')
        elif ag <= 20:
            df_class.append('<= 20')
        elif ag <= 25:
            df_class.append('<= 25')
        elif ag <= 30:
            df_class.append('<= 30')
        elif ag <= 35:
            df_class.append('<= 35')
        else:
            df_class.append('> 35')
    return df_class

df_daily_incidents['CLASS_AGING'] = class_aging(pd.to_numeric(df_daily_incidents['AGING']))

df_daily_incidents['BORDER_NAME'] = df_daily_incidents['BORDER'] + " - " + df_daily_incidents['ASSIGNED_TO']

In [15]:
df_daily_incidents = pd.merge(df_daily_incidents, df_inc_sla, left_on = 'NUMBER', right_on = 'NUMBER', how ='left')
df_daily_incidents['BUSINESS_PERCENTAGE'] = df_daily_incidents['BUSINESS_PERCENTAGE'].fillna(0)

In [16]:
# Country Grouped
df_country_grouped = pd.DataFrame({
})

# JOIN LOCATION
df_daily_incidents_merged = pd.merge(df_daily_incidents, df_country_grouped, left_on='LOCATION', right_on='LOCATION', how = 'left')

df_daily_incidents_merged['LOCATION GROUPED'][df_daily_incidents_merged['LOCATION GROUPED'].isna()] = 'OTHERS'

df_daily_incidents_merged['LOCATION'] = df_daily_incidents_merged['LOCATION'].str.upper()
df_daily_incidents_merged['LOCATION COUNTRY'] = df_daily_incidents_merged['LOCATION GROUPED'].str.upper()
df_daily_incidents_merged['LOCATION GROUPED'] = df_daily_incidents_merged['LOCATION GROUPED'].str.upper()

if os.path.exists(f''):
    os.remove(f'')

df_daily_incidents_merged = df_daily_incidents_merged[df_daily_incidents_merged['FLAG_L12M_REFERENCE']==True]

df_daily_incidents_merged.to_csv(rf'', sep=';', index=False)

#df_daily_incidents_merged[df_daily_incidents_merged['NUMBER']=='INC1078701']

#df_daily_incidents_merged[(pd.to_datetime(df_daily_incidents['REFERENCE_AT_DATE']).dt.date==current_date_dt)
#                   & (df_daily_incidents['STATUS_MONTH']=='BACKLOG')].shape

C:\Users\rafael.ferrari\AppData\Local\Temp\ipykernel_16460\2310580184.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_daily_incidents_merged['LOCATION GROUPED'][df_daily_incidents_merged['LOCATION GROUPED'].isna()] = 'OTHERS'
